# 连续处理效应 - 从二元到连续

## 业务场景引入

### 场景 1: 外卖平台优惠券金额优化
"我们想知道优惠券金额对用户下单的影响。5元券好还是10元券好？15元券会不会更好？"

### 场景 2: 广告投放预算决策
"增加1000元广告预算能带来多少额外收入？继续增加是否有边际递减效应？"

### 场景 3: 定价策略分析
"产品定价从99元调整到129元，销量会下降多少？最优价格点在哪里？"

### 核心挑战
- **二元处理**: 处理 vs 不处理 (送券 vs 不送券)
- **连续处理**: 处理强度变化 (5元 vs 10元 vs 15元...)
- **目标**: 估计**剂量响应函数** (Dose-Response Function)

---

## 本章学习目标
1. 理解连续处理与二元处理的本质区别
2. 掌握广义倾向得分 (GPS) 方法
3. 学习剂量响应函数的多种估计方法
4. 运用双重机器学习处理连续处理
5. 解决真实业务中的连续处理问题

In [ ]:
# 环境配置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from scipy.interpolate import UnivariateSpline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.kernel_ridge import KernelRidge
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 主题
colors = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0'
}

print("✅ 环境配置完成")

---

# Part 1: 连续处理的挑战

## 1.1 二元处理 vs 连续处理

### 二元处理 (Binary Treatment)
- 处理变量 $T \in \{0, 1\}$
- 估计目标: 平均处理效应 (ATE)
$$
\tau = E[Y(1) - Y(0)]
$$

### 连续处理 (Continuous Treatment)
- 处理变量 $T \in \mathbb{R}$ (或有界区间)
- 估计目标: 剂量响应函数 (DRF)
$$
\mu(t) = E[Y(t)]
$$

### 核心差异
| 维度 | 二元处理 | 连续处理 |
|------|----------|----------|
| 处理空间 | {0, 1} | 连续 |
| 估计目标 | 单个数值 (ATE) | 函数 $\mu(t)$ |
| 倾向得分 | $P(T=1\|X)$ | 条件密度 $f(T\|X)$ |
| 可识别性 | Overlap | Positivity |
| 复杂度 | 相对简单 | 更复杂 |

In [ ]:
def visualize_treatment_comparison():
    """可视化二元 vs 连续处理的区别"""
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('二元处理', '连续处理'),
        horizontal_spacing=0.15
    )
    
    # 二元处理
    fig.add_trace(
        go.Bar(
            x=['不处理 (T=0)', '处理 (T=1)'],
            y=[100, 120],
            marker_color=[colors['danger'], colors['success']],
            text=['E[Y(0)]=100', 'E[Y(1)]=120'],
            textposition='outside',
            name='期望结果'
        ),
        row=1, col=1
    )
    
    # 添加 ATE 箭头
    fig.add_annotation(
        x=0.5, y=110,
        text='ATE = 20',
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor=colors['primary'],
        ax=0, ay=-40,
        row=1, col=1,
        xref='x1', yref='y1'
    )
    
    # 连续处理 - 剂量响应曲线
    t_values = np.linspace(0, 20, 100)
    # 真实 DRF: 对数形式 (边际递减)
    drf = 100 + 15 * np.log(t_values + 1)
    
    fig.add_trace(
        go.Scatter(
            x=t_values,
            y=drf,
            mode='lines',
            line=dict(color=colors['primary'], width=3),
            name='剂量响应函数',
            fill='tozeroy',
            fillcolor='rgba(45, 156, 219, 0.1)'
        ),
        row=1, col=2
    )
    
    # 标注几个关键点
    key_points = [0, 5, 10, 15, 20]
    for t in key_points:
        y = 100 + 15 * np.log(t + 1)
        fig.add_trace(
            go.Scatter(
                x=[t], y=[y],
                mode='markers',
                marker=dict(size=10, color=colors['success']),
                showlegend=False,
                hovertemplate=f'T={t}<br>E[Y({t})]={y:.1f}<extra></extra>'
            ),
            row=1, col=2
        )
    
    fig.update_xaxes(title_text="处理状态", row=1, col=1)
    fig.update_xaxes(title_text="处理剂量 (T)", row=1, col=2)
    fig.update_yaxes(title_text="期望结果", row=1, col=1)
    fig.update_yaxes(title_text="期望结果 E[Y(t)]", row=1, col=2)
    
    fig.update_layout(
        height=400,
        showlegend=False,
        title_text="二元处理 vs 连续处理对比",
        template='plotly_white'
    )
    
    return fig

visualize_treatment_comparison().show()

## 1.2 剂量响应函数 (Dose-Response Function)

### 定义
剂量响应函数描述了处理强度与潜在结果之间的关系:
$$
\mu(t) = E[Y(t)]
$$

### 常见形式

#### 1. 线性响应
$$
\mu(t) = \beta_0 + \beta_1 t
$$
- 简单但可能过于限制性
- 假设边际效应恒定

#### 2. 对数响应 (边际递减)
$$
\mu(t) = \beta_0 + \beta_1 \log(t + 1)
$$
- 广告投放常见
- 边际效应递减

#### 3. 多项式响应
$$
\mu(t) = \beta_0 + \beta_1 t + \beta_2 t^2 + \ldots
$$
- 可以捕捉非单调关系
- 可能存在最优剂量点

#### 4. 阈值效应
$$
\mu(t) = \begin{cases}
\beta_0 & t < t_0 \\
\beta_0 + \beta_1(t - t_0) & t \geq t_0
\end{cases}
$$
- 存在最小有效剂量
- 药物剂量常见

In [ ]:
def plot_drf_forms():
    """可视化不同形式的剂量响应函数"""
    
    t = np.linspace(0, 20, 200)
    
    # 不同的 DRF 形式
    drfs = {
        '线性响应': 100 + 2 * t,
        '对数响应 (边际递减)': 100 + 20 * np.log(t + 1),
        '二次响应 (存在最优点)': 100 + 8*t - 0.2*t**2,
        'S型响应 (Sigmoid)': 100 + 40 / (1 + np.exp(-0.5*(t-10)))
    }
    
    fig = go.Figure()
    
    colors_list = [colors['primary'], colors['success'], colors['danger'], colors['warning']]
    
    for (name, drf_values), color in zip(drfs.items(), colors_list):
        fig.add_trace(go.Scatter(
            x=t,
            y=drf_values,
            mode='lines',
            name=name,
            line=dict(width=3, color=color)
        ))
    
    fig.update_layout(
        title='不同形式的剂量响应函数',
        xaxis_title='处理剂量 (T)',
        yaxis_title='期望结果 E[Y(t)]',
        height=500,
        template='plotly_white',
        hovermode='x unified'
    )
    
    return fig

plot_drf_forms().show()

## 1.3 应用场景

### 互联网行业
1. **优惠券金额优化**: 5元、10元、15元券的转化效果
2. **广告预算分配**: 投放金额与转化的关系
3. **推荐位数量**: 展示1个、3个、5个推荐的效果
4. **视频时长**: 15s、30s、60s广告的完播率

### 医疗健康
1. **药物剂量**: 剂量与治愈率的关系
2. **运动时长**: 每日运动时间与健康指标
3. **营养摄入**: 维生素摄入量与健康状况

### 经济金融
1. **利率政策**: 利率水平与经济增长
2. **补贴力度**: 政府补贴金额与企业发展
3. **税率调整**: 税率与税收收入的关系

---

# Part 2: 广义倾向得分 (Generalized Propensity Score)

## 2.1 从倾向得分到广义倾向得分

### 二元处理的倾向得分
$$
e(X) = P(T=1 | X)
$$

### 连续处理的广义倾向得分 (GPS)
$$
r(t, X) = f_T(t | X)
$$
其中 $f_T(t | X)$ 是给定协变量 $X$ 下处理变量 $T$ 的**条件密度函数**。

### GPS 的核心性质

#### 1. 平衡性 (Balancing Property)
$$
X \perp T | r(T, X)
$$
给定 GPS，协变量与处理独立。

#### 2. 可识别性
如果满足:
- **无混淆假设**: $Y(t) \perp T | X$
- **正性假设**: $f_T(t|X) > 0$ for all $t$ in support of $T$

则剂量响应函数可识别:
$$
\mu(t) = E\left[E[Y | T=t, r(t, X)]\right]
$$

## 2.2 GPS 的估计

### 参数方法: 假设正态分布
假设给定 $X$，处理变量服从正态分布:
$$
T | X \sim N(\alpha_0 + X'\alpha, \sigma^2)
$$

则 GPS 为:
$$
r(t, X) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(t - \alpha_0 - X'\alpha)^2}{2\sigma^2}\right)
$$

**估计步骤**:
1. 回归 $T$ 对 $X$: 得到 $\hat{\alpha}, \hat{\sigma}^2$
2. 计算每个观测的 GPS: $\hat{r}(T_i, X_i)$

In [ ]:
class GeneralizedPropensityScore:
    """
    广义倾向得分估计器
    
    假设: T | X ~ N(X'α, σ²)
    """
    
    def __init__(self):
        self.model = LinearRegression()
        self.sigma = None
        
    def fit(self, T, X):
        """
        拟合 GPS 模型
        
        Parameters:
        -----------
        T : array-like, shape (n,)
            处理变量
        X : array-like, shape (n, p)
            协变量
        """
        # 步骤 1: 回归 T ~ X
        self.model.fit(X, T)
        
        # 步骤 2: 计算残差标准差
        residuals = T - self.model.predict(X)
        self.sigma = np.std(residuals)
        
        return self
    
    def predict(self, T, X):
        """
        计算 GPS: r(t, X)
        
        Returns:
        --------
        gps : array-like, shape (n,)
            广义倾向得分
        """
        # 预测 E[T|X]
        T_pred = self.model.predict(X)
        
        # 计算正态密度
        gps = stats.norm.pdf(T, loc=T_pred, scale=self.sigma)
        
        return gps
    
    def check_balance(self, T, X, n_bins=5):
        """
        检查 GPS 的平衡性
        
        在不同 GPS 区间内，检查协变量的均值是否相似
        """
        gps = self.predict(T, X)
        
        # 将 GPS 分成 bins
        gps_bins = pd.qcut(gps, q=n_bins, labels=False, duplicates='drop')
        
        balance_results = []
        
        for j in range(X.shape[1]):
            x_j = X[:, j]
            
            # 计算每个 bin 内的均值
            bin_means = [x_j[gps_bins == i].mean() for i in range(n_bins)]
            
            # 标准化差异
            overall_std = x_j.std()
            max_diff = (max(bin_means) - min(bin_means)) / overall_std
            
            balance_results.append({
                'covariate': f'X{j+1}',
                'max_std_diff': max_diff
            })
        
        return pd.DataFrame(balance_results)

print("✅ GPS 估计器定义完成")

## 2.3 数据生成与 GPS 估计示例

In [ ]:
def generate_continuous_treatment_data(n=2000, seed=42):
    """
    生成连续处理数据
    
    DGP:
    - X1, X2 ~ N(0, 1)
    - T | X ~ N(X1 + 0.5*X2, 1)
    - Y(t) = 100 + 2*X1 - X2 + 3*T - 0.1*T² + ε
    """
    np.random.seed(seed)
    
    # 协变量
    X1 = np.random.normal(0, 1, n)
    X2 = np.random.normal(0, 1, n)
    X = np.column_stack([X1, X2])
    
    # 处理变量 (有混淆)
    T_mean = X1 + 0.5 * X2
    T = np.random.normal(T_mean, 1)
    # 截断到 [0, 10]
    T = np.clip(T, 0, 10)
    
    # 结果变量 (二次 DRF)
    Y = 100 + 2*X1 - X2 + 3*T - 0.1*T**2 + np.random.normal(0, 0.5, n)
    
    # 真实 DRF
    def true_drf(t):
        # E[Y(t)] = E[100 + 2*X1 - X2 + 3*t - 0.1*t²]
        #         = 100 + 3*t - 0.1*t² (因为 E[X1]=E[X2]=0)
        return 100 + 3*t - 0.1*t**2
    
    return {
        'X': X,
        'T': T,
        'Y': Y,
        'true_drf': true_drf
    }

# 生成数据
data = generate_continuous_treatment_data()
X, T, Y = data['X'], data['T'], data['Y']

print(f"数据维度: n={len(T)}, p={X.shape[1]}")
print(f"处理范围: [{T.min():.2f}, {T.max():.2f}]")
print(f"结果均值: {Y.mean():.2f}")

In [ ]:
# 估计 GPS
gps_estimator = GeneralizedPropensityScore()
gps_estimator.fit(T, X)

# 计算 GPS
gps_values = gps_estimator.predict(T, X)

print(f"GPS 均值: {gps_values.mean():.4f}")
print(f"GPS 范围: [{gps_values.min():.4f}, {gps_values.max():.4f}]")
print(f"估计的 σ: {gps_estimator.sigma:.4f}")

In [ ]:
def plot_gps_distribution():
    """可视化 GPS 分布与处理分布"""
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('处理变量分布', 'GPS 分布'),
        horizontal_spacing=0.12
    )
    
    # 处理变量分布
    fig.add_trace(
        go.Histogram(
            x=T,
            nbinsx=50,
            marker_color=colors['primary'],
            name='观测分布',
            opacity=0.7
        ),
        row=1, col=1
    )
    
    # GPS 分布
    fig.add_trace(
        go.Histogram(
            x=gps_values,
            nbinsx=50,
            marker_color=colors['success'],
            name='GPS 分布',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="处理剂量 (T)", row=1, col=1)
    fig.update_xaxes(title_text="GPS 值", row=1, col=2)
    fig.update_yaxes(title_text="频数", row=1, col=1)
    fig.update_yaxes(title_text="频数", row=1, col=2)
    
    fig.update_layout(
        height=400,
        showlegend=False,
        template='plotly_white'
    )
    
    return fig

plot_gps_distribution().show()

In [ ]:
# 平衡性检验
balance_check = gps_estimator.check_balance(T, X, n_bins=5)

print("\n平衡性检验结果:")
print(balance_check)
print("\n判断标准: 标准化差异 < 0.1 表示良好平衡")

---

# Part 3: 剂量响应函数估计

## 3.1 基于 GPS 的估计方法

### Hirano & Imbens (2004) 方法

**步骤 1**: 估计 GPS
$$
\hat{r}(T_i, X_i) = \hat{f}_T(T_i | X_i)
$$

**步骤 2**: 估计条件期望函数
$$
\beta(t, r) = E[Y | T=t, r(t, X)=r]
$$
使用回归:
$$
Y_i = \beta_0 + \beta_1 T_i + \beta_2 T_i^2 + \beta_3 \hat{r}_i + \beta_4 \hat{r}_i^2 + \ldots + \epsilon_i
$$

**步骤 3**: 边际化得到 DRF
$$
\hat{\mu}(t) = \frac{1}{n} \sum_{i=1}^n \hat{\beta}(t, \hat{r}(t, X_i))
$$

## 3.2 参数方法

In [ ]:
class GPS_DRF_Estimator:
    """
    基于 GPS 的剂量响应函数估计
    
    实现 Hirano & Imbens (2004) 方法
    """
    
    def __init__(self, treatment_degree=2, gps_degree=2):
        """
        Parameters:
        -----------
        treatment_degree : int
            处理变量的多项式阶数
        gps_degree : int
            GPS 的多项式阶数
        """
        self.treatment_degree = treatment_degree
        self.gps_degree = gps_degree
        self.gps_estimator = GeneralizedPropensityScore()
        self.outcome_model = None
        
    def fit(self, T, X, Y):
        """
        步骤 1: 估计 GPS
        步骤 2: 估计条件期望函数 E[Y|T,r]
        """
        # 步骤 1: GPS
        self.gps_estimator.fit(T, X)
        gps = self.gps_estimator.predict(T, X)
        
        # 步骤 2: 构造特征 (T, T², r, r²)
        features = []
        
        # 处理变量的多项式
        for d in range(1, self.treatment_degree + 1):
            features.append(T ** d)
        
        # GPS 的多项式
        for d in range(1, self.gps_degree + 1):
            features.append(gps ** d)
        
        # 交互项
        features.append(T * gps)
        
        Z = np.column_stack(features)
        
        # 拟合结果模型
        self.outcome_model = LinearRegression()
        self.outcome_model.fit(Z, Y)
        
        return self
    
    def predict_drf(self, t_grid, X):
        """
        步骤 3: 估计 DRF μ(t)
        
        Parameters:
        -----------
        t_grid : array-like
            要估计的处理水平网格
        X : array-like
            协变量 (用于计算 GPS)
        
        Returns:
        --------
        drf : array-like
            每个 t 的估计值
        """
        drf_estimates = []
        
        for t in t_grid:
            # 对每个个体计算 E[Y|T=t, r(t,Xi)]
            T_temp = np.full(len(X), t)
            gps_temp = self.gps_estimator.predict(T_temp, X)
            
            # 构造特征
            features = []
            for d in range(1, self.treatment_degree + 1):
                features.append(T_temp ** d)
            for d in range(1, self.gps_degree + 1):
                features.append(gps_temp ** d)
            features.append(T_temp * gps_temp)
            
            Z_temp = np.column_stack(features)
            
            # 预测
            Y_pred = self.outcome_model.predict(Z_temp)
            
            # 边际化: 取均值
            drf_estimates.append(Y_pred.mean())
        
        return np.array(drf_estimates)

print("✅ GPS-DRF 估计器定义完成")

In [ ]:
# 估计 DRF
gps_drf = GPS_DRF_Estimator(treatment_degree=2, gps_degree=2)
gps_drf.fit(T, X, Y)

# 预测网格
t_grid = np.linspace(0, 10, 50)
drf_est = gps_drf.predict_drf(t_grid, X)

# 真实 DRF
drf_true = data['true_drf'](t_grid)

# 可视化
fig = go.Figure()

# 真实 DRF
fig.add_trace(go.Scatter(
    x=t_grid,
    y=drf_true,
    mode='lines',
    name='真实 DRF',
    line=dict(color=colors['success'], width=3, dash='dash')
))

# 估计 DRF
fig.add_trace(go.Scatter(
    x=t_grid,
    y=drf_est,
    mode='lines',
    name='GPS 估计',
    line=dict(color=colors['primary'], width=3)
))

# 添加观测数据的散点 (采样展示)
sample_idx = np.random.choice(len(T), 200, replace=False)
fig.add_trace(go.Scatter(
    x=T[sample_idx],
    y=Y[sample_idx],
    mode='markers',
    name='观测数据',
    marker=dict(size=3, color=colors['danger'], opacity=0.3)
))

fig.update_layout(
    title='剂量响应函数估计 (GPS 方法)',
    xaxis_title='处理剂量 (T)',
    yaxis_title='期望结果 E[Y(t)]',
    height=500,
    template='plotly_white'
)

fig.show()

# 计算 MSE
mse = np.mean((drf_est - drf_true) ** 2)
print(f"\nDRF 估计 MSE: {mse:.4f}")

## 3.3 非参数方法: 局部多项式回归

不假设 DRF 的函数形式，使用核平滑方法:

$$
\hat{\mu}(t) = \frac{\sum_{i=1}^n K_h(T_i - t) \cdot Y_i}{\sum_{i=1}^n K_h(T_i - t)}
$$

其中 $K_h(u) = \frac{1}{h} K(\frac{u}{h})$ 是核函数，$h$ 是带宽。

### 需要调整混淆
加入 GPS 权重:
$$
\hat{\mu}(t) = \frac{\sum_{i=1}^n K_h(T_i - t) \cdot w_i \cdot Y_i}{\sum_{i=1}^n K_h(T_i - t) \cdot w_i}
$$

权重可以是:
$$
w_i = \frac{1}{\hat{r}(T_i, X_i)}
$$

In [ ]:
def kernel_drf_estimation(T, Y, gps, t_grid, bandwidth='scott'):
    """
    基于核平滑的 DRF 估计
    
    使用 GPS 逆概率加权
    """
    from scipy.stats import gaussian_kde
    
    # 计算权重
    weights = 1.0 / gps
    # 归一化
    weights = weights / weights.sum() * len(weights)
    
    drf_estimates = []
    
    for t in t_grid:
        # 高斯核
        if bandwidth == 'scott':
            h = 1.06 * np.std(T) * len(T) ** (-1/5)
        else:
            h = bandwidth
        
        # 核权重
        kernel_weights = np.exp(-0.5 * ((T - t) / h) ** 2) / (h * np.sqrt(2 * np.pi))
        
        # 组合权重
        combined_weights = kernel_weights * weights
        
        # 加权平均
        if combined_weights.sum() > 0:
            drf_t = np.sum(combined_weights * Y) / np.sum(combined_weights)
        else:
            drf_t = np.nan
        
        drf_estimates.append(drf_t)
    
    return np.array(drf_estimates)

# 非参数估计
drf_nonparam = kernel_drf_estimation(T, Y, gps_values, t_grid, bandwidth=0.5)

# 对比可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t_grid, y=drf_true,
    mode='lines',
    name='真实 DRF',
    line=dict(color='black', width=3, dash='dash')
))

fig.add_trace(go.Scatter(
    x=t_grid, y=drf_est,
    mode='lines',
    name='参数方法 (GPS)',
    line=dict(color=colors['primary'], width=2)
))

fig.add_trace(go.Scatter(
    x=t_grid, y=drf_nonparam,
    mode='lines',
    name='非参数方法 (Kernel)',
    line=dict(color=colors['warning'], width=2)
))

fig.update_layout(
    title='DRF 估计方法对比',
    xaxis_title='处理剂量 (T)',
    yaxis_title='期望结果 E[Y(t)]',
    height=500,
    template='plotly_white'
)

fig.show()

print(f"参数方法 MSE: {np.mean((drf_est - drf_true)**2):.4f}")
print(f"非参数方法 MSE: {np.nanmean((drf_nonparam - drf_true)**2):.4f}")

### 💡 TODO 1: 实现样条回归估计 DRF

使用三次样条 (Cubic Spline) 拟合剂量响应函数。

提示:
1. 使用 `scipy.interpolate.UnivariateSpline`
2. 先用 GPS 调整偏差 (可以用 GPS 加权)
3. 对调整后的数据拟合样条
4. 在 `t_grid` 上预测

In [ ]:
# TODO 1: 实现样条回归 DRF 估计

def spline_drf_estimation(T, Y, gps, t_grid, s=None):
    """
    基于样条回归的 DRF 估计
    
    Parameters:
    -----------
    T : array-like
        处理变量
    Y : array-like
        结果变量
    gps : array-like
        广义倾向得分
    t_grid : array-like
        预测网格
    s : float or None
        平滑参数 (None 表示自动选择)
    
    Returns:
    --------
    drf_estimates : array-like
    """
    # 你的代码:
    # 1. 计算 GPS 权重
    weights = 1.0 / gps
    
    # 2. 排序 (样条需要有序输入)
    sorted_idx = np.argsort(T)
    T_sorted = T[sorted_idx]
    Y_sorted = Y[sorted_idx]
    weights_sorted = weights[sorted_idx]
    
    # 3. 拟合样条
    spline = UnivariateSpline(T_sorted, Y_sorted, w=weights_sorted, s=s, k=3)
    
    # 4. 预测
    drf_estimates = spline(t_grid)
    
    return drf_estimates

# 测试
drf_spline = spline_drf_estimation(T, Y, gps_values, t_grid, s=1000)

# 可视化
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_grid, y=drf_true, mode='lines', name='真实', line=dict(dash='dash', color='black', width=3)))
fig.add_trace(go.Scatter(x=t_grid, y=drf_spline, mode='lines', name='样条估计', line=dict(color=colors['info'], width=2)))
fig.update_layout(title='样条回归 DRF 估计', xaxis_title='T', yaxis_title='E[Y(t)]', template='plotly_white')
fig.show()

print(f"样条方法 MSE: {np.mean((drf_spline - drf_true)**2):.4f}")

---

# Part 4: 双重机器学习 (DML) for 连续处理

## 4.1 DML 框架回顾

双重机器学习通过**正交化**减少模型误设风险。

### 核心思想
将目标参数与干扰参数分离:
$$
Y = \theta(T) + g(X) + \epsilon
$$
$$
T = m(X) + \eta
$$

**目标**: 估计 $\theta(t)$ (剂量响应函数)
**干扰**: $g(X)$ 和 $m(X)$

## 4.2 连续处理的 DML

### Robinson (1988) 的部分线性模型
假设:
$$
Y = \theta(T) + g(X) + \epsilon, \quad E[\epsilon|T,X] = 0
$$
$$
T = m(X) + \eta, \quad E[\eta|X] = 0
$$

### DML 估计步骤

**步骤 1**: 预测 $E[Y|X]$ 和 $E[T|X]$
- 用机器学习模型: $\hat{g}(X), \hat{m}(X)$

**步骤 2**: 计算残差
$$
\tilde{Y}_i = Y_i - \hat{g}(X_i)
$$
$$
\tilde{T}_i = T_i - \hat{m}(X_i)
$$

**步骤 3**: 回归残差
$$
\tilde{Y}_i = \theta(\tilde{T}_i) + \epsilon_i
$$

### 交叉拟合 (Cross-Fitting)
为了避免过拟合偏差，使用 K 折交叉:
- 在折 $k$ 上训练 $\hat{g}, \hat{m}$
- 在其余折上预测残差
- 汇总所有折的残差进行最终估计

In [ ]:
from sklearn.model_selection import KFold

class DML_ContinuousTreatment:
    """
    双重机器学习 - 连续处理版本
    
    实现 Robinson (1988) 的部分线性模型
    """
    
    def __init__(self, model_y=None, model_t=None, drf_degree=2, n_splits=5):
        """
        Parameters:
        -----------
        model_y : estimator
            E[Y|X] 的预测模型
        model_t : estimator
            E[T|X] 的预测模型
        drf_degree : int
            DRF 的多项式阶数
        n_splits : int
            交叉拟合的折数
        """
        self.model_y = model_y or RandomForestRegressor(n_estimators=100, random_state=42)
        self.model_t = model_t or RandomForestRegressor(n_estimators=100, random_state=42)
        self.drf_degree = drf_degree
        self.n_splits = n_splits
        self.drf_model = None
        
    def fit(self, T, X, Y):
        """
        DML 估计
        """
        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        
        # 存储残差
        Y_residual = np.zeros(len(Y))
        T_residual = np.zeros(len(T))
        
        for train_idx, test_idx in kf.split(X):
            # 训练集
            X_train, T_train, Y_train = X[train_idx], T[train_idx], Y[train_idx]
            X_test, T_test, Y_test = X[test_idx], T[test_idx], Y[test_idx]
            
            # 步骤 1: 预测 E[Y|X] 和 E[T|X]
            model_y_fold = clone(self.model_y)
            model_t_fold = clone(self.model_t)
            
            model_y_fold.fit(X_train, Y_train)
            model_t_fold.fit(X_train, T_train)
            
            # 步骤 2: 计算测试集残差
            Y_residual[test_idx] = Y_test - model_y_fold.predict(X_test)
            T_residual[test_idx] = T_test - model_t_fold.predict(X_test)
        
        # 步骤 3: 用残差拟合 DRF
        # 使用多项式特征
        poly = PolynomialFeatures(degree=self.drf_degree, include_bias=True)
        T_residual_poly = poly.fit_transform(T_residual.reshape(-1, 1))
        
        self.drf_model = LinearRegression()
        self.drf_model.fit(T_residual_poly, Y_residual)
        
        self.poly = poly
        
        return self
    
    def predict_drf(self, t_grid):
        """
        预测 DRF
        
        注意: DML 估计的是去除 X 影响后的 DRF
        """
        t_grid_poly = self.poly.transform(t_grid.reshape(-1, 1))
        return self.drf_model.predict(t_grid_poly)

def clone(model):
    """简单的模型克隆"""
    from copy import deepcopy
    return deepcopy(model)

print("✅ DML 估计器定义完成")

In [ ]:
# DML 估计
dml = DML_ContinuousTreatment(
    model_y=GradientBoostingRegressor(n_estimators=100, random_state=42),
    model_t=GradientBoostingRegressor(n_estimators=100, random_state=42),
    drf_degree=2,
    n_splits=5
)

dml.fit(T, X, Y)

# 预测 DRF
# 注意: DML 估计的是相对于基线的增量
# 需要加上基线 (T=0 时的期望)
drf_dml_increment = dml.predict_drf(t_grid)
baseline = Y.mean() - dml.predict_drf(T).mean()  # 估计基线
drf_dml = drf_dml_increment + baseline

# 可视化对比
fig = go.Figure()

fig.add_trace(go.Scatter(x=t_grid, y=drf_true, mode='lines', name='真实 DRF', 
                         line=dict(color='black', width=3, dash='dash')))
fig.add_trace(go.Scatter(x=t_grid, y=drf_est, mode='lines', name='GPS 方法',
                         line=dict(color=colors['primary'], width=2)))
fig.add_trace(go.Scatter(x=t_grid, y=drf_dml, mode='lines', name='DML 方法',
                         line=dict(color=colors['success'], width=2)))

fig.update_layout(
    title='DRF 估计: GPS vs DML',
    xaxis_title='处理剂量 (T)',
    yaxis_title='期望结果 E[Y(t)]',
    height=500,
    template='plotly_white'
)

fig.show()

print(f"GPS 方法 MSE: {np.mean((drf_est - drf_true)**2):.4f}")
print(f"DML 方法 MSE: {np.mean((drf_dml - drf_true)**2):.4f}")

---

# Part 5: 业务案例

## 案例 1: 优惠券金额优化

### 业务背景
某外卖平台想优化优惠券金额策略。历史数据显示:
- 优惠券金额范围: 0-30元
- 用户特征: 过往消费额、活跃度
- 目标: 订单金额最大化

### 核心问题
1. 优惠券金额与订单金额的关系?
2. 存在最优券额吗?
3. 是否所有用户都适用同一券额?

In [ ]:
def generate_coupon_data(n=3000, seed=42):
    """
    生成优惠券案例数据
    
    DGP:
    - X1: 过往消费额 (标准化)
    - X2: 活跃度 (标准化)
    - T: 券额 (0-30元)
    - Y: 订单金额
    
    真实 DRF (二次, 存在最优点):
    E[Y(t)] = 80 + 10*t - 0.2*t²
    最优券额: t* = 10/0.4 = 25元
    """
    np.random.seed(seed)
    
    # 用户特征
    past_spending = np.random.gamma(2, 2, n)  # 过往消费
    activity = np.random.beta(2, 5, n)        # 活跃度
    
    # 标准化
    X1 = (past_spending - past_spending.mean()) / past_spending.std()
    X2 = (activity - activity.mean()) / activity.std()
    X = np.column_stack([X1, X2])
    
    # 券额分配 (有混淆: 高消费用户倾向获得大券)
    T_mean = 15 + 5 * X1 + 3 * X2
    T = np.random.normal(T_mean, 3)
    T = np.clip(T, 0, 30)
    
    # 订单金额
    # 基础: 与用户特征相关
    base_order = 50 + 20 * X1 + 10 * X2
    # 券额效应 (二次)
    coupon_effect = 10 * T - 0.2 * T**2
    # 噪音
    Y = base_order + coupon_effect + np.random.normal(0, 5, n)
    
    # 真实 DRF (边际化 X)
    def true_drf(t):
        return 50 + 10*t - 0.2*t**2  # E[base_order]=50
    
    return {
        'X': X,
        'T': T,
        'Y': Y,
        'feature_names': ['过往消费', '活跃度'],
        'true_drf': true_drf,
        'optimal_coupon': 25.0
    }

coupon_data = generate_coupon_data()

print("优惠券数据生成完成")
print(f"样本量: {len(coupon_data['T'])}")
print(f"券额范围: [{coupon_data['T'].min():.1f}, {coupon_data['T'].max():.1f}]")
print(f"订单金额均值: {coupon_data['Y'].mean():.2f}")
print(f"理论最优券额: {coupon_data['optimal_coupon']:.1f} 元")

In [ ]:
# 使用 DML 估计 DRF
dml_coupon = DML_ContinuousTreatment(
    model_y=GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42),
    model_t=GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42),
    drf_degree=3,  # 使用3次多项式捕捉曲率
    n_splits=5
)

dml_coupon.fit(coupon_data['T'], coupon_data['X'], coupon_data['Y'])

# 预测 DRF
coupon_grid = np.linspace(0, 30, 100)
drf_coupon_increment = dml_coupon.predict_drf(coupon_grid)
baseline_coupon = coupon_data['Y'].mean() - dml_coupon.predict_drf(coupon_data['T']).mean()
drf_coupon_est = drf_coupon_increment + baseline_coupon

# 真实 DRF
drf_coupon_true = coupon_data['true_drf'](coupon_grid)

# 找到估计的最优券额
optimal_idx = np.argmax(drf_coupon_est)
optimal_coupon_est = coupon_grid[optimal_idx]
max_order_value_est = drf_coupon_est[optimal_idx]

# 可视化
fig = go.Figure()

# 真实 DRF
fig.add_trace(go.Scatter(
    x=coupon_grid,
    y=drf_coupon_true,
    mode='lines',
    name='真实关系',
    line=dict(color='black', width=3, dash='dash')
))

# 估计 DRF
fig.add_trace(go.Scatter(
    x=coupon_grid,
    y=drf_coupon_est,
    mode='lines',
    name='DML 估计',
    line=dict(color=colors['primary'], width=3),
    fill='tozeroy',
    fillcolor='rgba(45, 156, 219, 0.1)'
))

# 标注最优点
fig.add_trace(go.Scatter(
    x=[optimal_coupon_est],
    y=[max_order_value_est],
    mode='markers',
    name=f'最优券额: {optimal_coupon_est:.1f}元',
    marker=dict(size=15, color=colors['danger'], symbol='star')
))

# 添加真实最优点
true_optimal_value = coupon_data['true_drf'](coupon_data['optimal_coupon'])
fig.add_trace(go.Scatter(
    x=[coupon_data['optimal_coupon']],
    y=[true_optimal_value],
    mode='markers',
    name=f'真实最优: {coupon_data["optimal_coupon"]:.1f}元',
    marker=dict(size=15, color=colors['success'], symbol='diamond')
))

fig.update_layout(
    title='优惠券金额优化 - 剂量响应曲线',
    xaxis_title='优惠券金额 (元)',
    yaxis_title='预期订单金额 (元)',
    height=500,
    template='plotly_white'
)

fig.show()

print(f"\n📊 优化结果:")
print(f"估计最优券额: {optimal_coupon_est:.1f} 元")
print(f"真实最优券额: {coupon_data['optimal_coupon']:.1f} 元")
print(f"估计最大订单额: {max_order_value_est:.2f} 元")
print(f"\n💡 业务建议: 发放约 {optimal_coupon_est:.0f} 元券可最大化订单金额")

## 案例 2: 广告预算的边际收益

### 业务背景
某电商平台评估广告投放预算的效果:
- 预算范围: 0-100万元/月
- 目标: 新增 GMV

### 核心问题
1. 每增加1万元预算,GMV 增加多少?
2. 边际收益是否递减?
3. ROI 最高的预算区间?

In [ ]:
def generate_ad_budget_data(n=1000, seed=42):
    """
    生成广告预算案例数据
    
    真实 DRF: 对数形式 (边际递减)
    E[Y(t)] = 500 + 200 * log(t + 1)
    """
    np.random.seed(seed)
    
    # 协变量: 产品类目、季节性
    category = np.random.choice([0, 1, 2], n)  # 3个类目
    seasonality = np.random.normal(0, 1, n)
    
    X1 = (category - 1) / 1.0  # 标准化
    X2 = seasonality
    X = np.column_stack([X1, X2])
    
    # 预算分配 (混淆: 高类目倾向高预算)
    T_mean = 30 + 15 * X1 + 5 * X2
    T = np.random.gamma(shape=4, scale=T_mean/4)  # 预算不能为负
    T = np.clip(T, 0, 100)
    
    # GMV (对数响应)
    base_gmv = 400 + 50 * X1 + 20 * X2
    ad_effect = 200 * np.log(T + 1)
    Y = base_gmv + ad_effect + np.random.normal(0, 30, n)
    
    def true_drf(t):
        return 400 + 200 * np.log(t + 1)
    
    return {
        'X': X,
        'T': T,
        'Y': Y,
        'true_drf': true_drf
    }

ad_data = generate_ad_budget_data()

print("广告预算数据生成完成")
print(f"样本量: {len(ad_data['T'])}")
print(f"预算范围: [{ad_data['T'].min():.1f}, {ad_data['T'].max():.1f}] 万元")
print(f"GMV 均值: {ad_data['Y'].mean():.2f} 万元")

### 💡 TODO 2: 估计广告预算的 DRF 并计算边际效应

任务:
1. 使用 GPS 或 DML 方法估计 DRF
2. 计算边际效应: $\frac{d\mu(t)}{dt}$
3. 计算 ROI: $\frac{d\mu(t)/dt}{1}$ (每增加1万元预算的 GMV 增量)
4. 可视化边际效应曲线

In [ ]:
# TODO 2: 估计 DRF 并计算边际效应

# 步骤 1: 估计 DRF
gps_ad = GPS_DRF_Estimator(treatment_degree=3, gps_degree=2)
gps_ad.fit(ad_data['T'], ad_data['X'], ad_data['Y'])

budget_grid = np.linspace(0, 100, 200)
drf_ad_est = gps_ad.predict_drf(budget_grid, ad_data['X'])
drf_ad_true = ad_data['true_drf'](budget_grid)

# 步骤 2: 计算边际效应 (数值导数)
def compute_marginal_effect(t_grid, drf_values):
    """计算边际效应: dμ/dt"""
    marginal = np.gradient(drf_values, t_grid)
    return marginal

marginal_est = compute_marginal_effect(budget_grid, drf_ad_est)
marginal_true = compute_marginal_effect(budget_grid, drf_ad_true)

# 步骤 3 & 4: 可视化
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('剂量响应曲线', '边际效应 (ROI)'),
    horizontal_spacing=0.12
)

# DRF
fig.add_trace(go.Scatter(x=budget_grid, y=drf_ad_true, mode='lines', name='真实',
                         line=dict(color='black', dash='dash', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=budget_grid, y=drf_ad_est, mode='lines', name='估计',
                         line=dict(color=colors['primary'], width=2)), row=1, col=1)

# 边际效应
fig.add_trace(go.Scatter(x=budget_grid, y=marginal_true, mode='lines', name='真实边际',
                         line=dict(color='black', dash='dash', width=2), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=budget_grid, y=marginal_est, mode='lines', name='估计边际',
                         line=dict(color=colors['success'], width=2), showlegend=False), row=1, col=2)

# 添加 ROI=1 的参考线
fig.add_hline(y=1, line_dash="dot", line_color="red", row=1, col=2,
              annotation_text="盈亏平衡点 (ROI=1)")

fig.update_xaxes(title_text="广告预算 (万元)", row=1, col=1)
fig.update_xaxes(title_text="广告预算 (万元)", row=1, col=2)
fig.update_yaxes(title_text="GMV (万元)", row=1, col=1)
fig.update_yaxes(title_text="边际 GMV (万元/万元预算)", row=1, col=2)

fig.update_layout(height=450, template='plotly_white', title_text='广告预算优化分析')
fig.show()

# 找到 ROI > 1 的最大预算
roi_positive = budget_grid[marginal_est > 1]
if len(roi_positive) > 0:
    max_positive_roi_budget = roi_positive.max()
    print(f"\n💡 业务建议: 预算在 {max_positive_roi_budget:.1f} 万元以下时 ROI > 1")
else:
    print("\n⚠️ 所有预算水平的 ROI 均 < 1")

## 案例 3: 定价策略分析

### 业务背景
某 SaaS 产品调整定价策略:
- 价格范围: 99-499 元/月
- 目标: 总收入 = 价格 × 转化率

### 挑战
价格影响两个维度:
1. **转化率**: 价格越高,转化率越低
2. **单价收入**: 价格越高,单个客户收入越高

需要找到收入最大化的价格点。

In [ ]:
def generate_pricing_data(n=2000, seed=42):
    """
    生成定价案例数据
    
    Y: 收入 = 价格 × 转化 indicator
    转化率 P(convert|price, X) 随价格下降
    """
    np.random.seed(seed)
    
    # 用户特征: 公司规模、行业
    company_size = np.random.lognormal(3, 1, n)  # 员工数
    industry = np.random.choice([0, 1, 2], n)    # 行业
    
    X1 = (np.log(company_size) - 3) / 1.0
    X2 = (industry - 1) / 1.0
    X = np.column_stack([X1, X2])
    
    # 价格分配 (混淆: 大公司倾向高价)
    T_mean = 250 + 50 * X1 + 30 * X2
    T = np.random.normal(T_mean, 50)
    T = np.clip(T, 99, 499)
    
    # 转化率 (logistic)
    logit_p = 2 + 0.5*X1 + 0.3*X2 - 0.01*T  # 价格越高,转化率越低
    prob_convert = 1 / (1 + np.exp(-logit_p))
    convert = np.random.binomial(1, prob_convert)
    
    # 收入 = 价格 × 转化
    Y = T * convert
    
    # 真实 DRF: E[Revenue(price)]
    def true_drf(t):
        # E[Y(t)] = t * E[convert(t)]
        # E[convert(t)] ≈ 1/(1 + exp(-2 + 0.01*t)) (边际化 X)
        logit_p = 2 - 0.01*t
        prob = 1 / (1 + np.exp(-logit_p))
        return t * prob
    
    return {
        'X': X,
        'T': T,
        'Y': Y,
        'convert': convert,
        'true_drf': true_drf
    }

pricing_data = generate_pricing_data()

print("定价数据生成完成")
print(f"样本量: {len(pricing_data['T'])}")
print(f"价格范围: [{pricing_data['T'].min():.0f}, {pricing_data['T'].max():.0f}] 元")
print(f"平均转化率: {pricing_data['convert'].mean():.2%}")
print(f"平均收入: {pricing_data['Y'].mean():.2f} 元")

### 💡 TODO 3: 定价优化

任务:
1. 估计价格-收入的 DRF
2. 找到收入最大化的价格点
3. 分析价格弹性

In [ ]:
# TODO 3: 定价优化

# 你的代码:
# 1. 估计 DRF (提示: 结果是收入,可能需要更灵活的模型)
dml_pricing = DML_ContinuousTreatment(
    model_y=GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42),
    model_t=GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42),
    drf_degree=3,
    n_splits=5
)

dml_pricing.fit(pricing_data['T'], pricing_data['X'], pricing_data['Y'])

# 2. 预测 DRF
price_grid = np.linspace(99, 499, 200)
drf_pricing_increment = dml_pricing.predict_drf(price_grid)
baseline_pricing = pricing_data['Y'].mean() - dml_pricing.predict_drf(pricing_data['T']).mean()
drf_pricing_est = drf_pricing_increment + baseline_pricing

# 确保非负
drf_pricing_est = np.maximum(drf_pricing_est, 0)

drf_pricing_true = pricing_data['true_drf'](price_grid)

# 3. 找到最优价格
optimal_price_idx = np.argmax(drf_pricing_est)
optimal_price = price_grid[optimal_price_idx]
max_revenue = drf_pricing_est[optimal_price_idx]

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=price_grid, y=drf_pricing_true,
    mode='lines', name='真实',
    line=dict(color='black', width=3, dash='dash')
))

fig.add_trace(go.Scatter(
    x=price_grid, y=drf_pricing_est,
    mode='lines', name='DML 估计',
    line=dict(color=colors['primary'], width=3),
    fill='tozeroy', fillcolor='rgba(45, 156, 219, 0.1)'
))

# 标注最优点
fig.add_trace(go.Scatter(
    x=[optimal_price], y=[max_revenue],
    mode='markers+text',
    name=f'最优价格',
    marker=dict(size=15, color=colors['danger'], symbol='star'),
    text=[f'{optimal_price:.0f}元<br>{max_revenue:.0f}元/客户'],
    textposition='top center'
))

fig.update_layout(
    title='定价优化 - 收入最大化',
    xaxis_title='价格 (元/月)',
    yaxis_title='预期收入 (元/客户)',
    height=500,
    template='plotly_white'
)

fig.show()

print(f"\n📊 优化结果:")
print(f"最优定价: {optimal_price:.0f} 元/月")
print(f"最大收入: {max_revenue:.2f} 元/客户")
print(f"\n对比当前平均价格 {pricing_data['T'].mean():.0f} 元:")
current_revenue_est = drf_pricing_est[np.argmin(np.abs(price_grid - pricing_data['T'].mean()))]
improvement = (max_revenue - current_revenue_est) / current_revenue_est * 100
print(f"收入提升潜力: {improvement:.1f}%")

---

# Part 6: 练习与思考题

## 练习 1: 方法对比

使用相同的数据,对比以下方法的性能:
1. GPS + 参数回归
2. GPS + 样条回归
3. DML (随机森林)
4. DML (梯度提升)

评估指标: MSE, 覆盖率

In [ ]:
# 练习 1: 方法对比

def compare_drf_methods(data, t_grid):
    """
    对比不同 DRF 估计方法
    """
    X, T, Y = data['X'], data['T'], data['Y']
    true_drf = data['true_drf'](t_grid)
    
    results = {}
    
    # 方法 1: GPS 参数
    gps1 = GPS_DRF_Estimator(treatment_degree=2, gps_degree=2)
    gps1.fit(T, X, Y)
    drf1 = gps1.predict_drf(t_grid, X)
    results['GPS 参数'] = {'drf': drf1, 'mse': np.mean((drf1 - true_drf)**2)}
    
    # 方法 2: GPS 样条
    gps_est = GeneralizedPropensityScore().fit(T, X)
    gps_vals = gps_est.predict(T, X)
    drf2 = spline_drf_estimation(T, Y, gps_vals, t_grid, s=1000)
    results['GPS 样条'] = {'drf': drf2, 'mse': np.mean((drf2 - true_drf)**2)}
    
    # 方法 3: DML (RF)
    dml_rf = DML_ContinuousTreatment(
        model_y=RandomForestRegressor(n_estimators=100, random_state=42),
        model_t=RandomForestRegressor(n_estimators=100, random_state=42),
        drf_degree=2
    )
    dml_rf.fit(T, X, Y)
    drf3_inc = dml_rf.predict_drf(t_grid)
    baseline3 = Y.mean() - dml_rf.predict_drf(T).mean()
    drf3 = drf3_inc + baseline3
    results['DML (RF)'] = {'drf': drf3, 'mse': np.mean((drf3 - true_drf)**2)}
    
    # 方法 4: DML (GBM)
    dml_gbm = DML_ContinuousTreatment(
        model_y=GradientBoostingRegressor(n_estimators=100, random_state=42),
        model_t=GradientBoostingRegressor(n_estimators=100, random_state=42),
        drf_degree=2
    )
    dml_gbm.fit(T, X, Y)
    drf4_inc = dml_gbm.predict_drf(t_grid)
    baseline4 = Y.mean() - dml_gbm.predict_drf(T).mean()
    drf4 = drf4_inc + baseline4
    results['DML (GBM)'] = {'drf': drf4, 'mse': np.mean((drf4 - true_drf)**2)}
    
    return results

# 运行对比
comparison = compare_drf_methods(data, t_grid)

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(x=t_grid, y=data['true_drf'](t_grid), mode='lines',
                         name='真实', line=dict(color='black', width=3, dash='dash')))

colors_comp = [colors['primary'], colors['success'], colors['warning'], colors['info']]
for (name, result), color in zip(comparison.items(), colors_comp):
    fig.add_trace(go.Scatter(x=t_grid, y=result['drf'], mode='lines',
                             name=f"{name} (MSE={result['mse']:.3f})",
                             line=dict(color=color, width=2)))

fig.update_layout(
    title='DRF 估计方法对比',
    xaxis_title='处理剂量',
    yaxis_title='期望结果',
    height=500,
    template='plotly_white'
)

fig.show()

# 排名
ranking = sorted(comparison.items(), key=lambda x: x[1]['mse'])
print("\n📊 方法排名 (按 MSE):")
for i, (name, result) in enumerate(ranking, 1):
    print(f"{i}. {name}: MSE = {result['mse']:.4f}")

## 思考题

### 思考题 1: GPS 的必要性
在什么情况下,我们可以忽略 GPS,直接对 $(T, Y)$ 进行回归?

<details>
<summary>答案提示</summary>

当处理分配是**完全随机**时,即 $T \perp X$,则:
- 没有混淆偏差
- 可以直接回归 $Y \sim T$
- GPS 调整不会改善估计

例如: 严格的随机实验 (RCT) 数据。
</details>

### 思考题 2: 最优剂量的置信区间
我们估计出最优券额是 25元,但这个估计有不确定性。如何构建最优剂量的置信区间?

<details>
<summary>答案提示</summary>

方法:
1. **Bootstrap**: 重采样数据,多次估计 DRF,记录每次的最优点
2. **Delta Method**: 如果 DRF 是参数模型,用 Delta 方法推导最优点的渐近分布
3. **Subsampling**: 子样本方法

Bootstrap 最简单实用。
</details>

### 思考题 3: 离散 vs 连续
假设优惠券只能是 {5, 10, 15, 20, 25} 这5个离散值,我们还需要估计连续的 DRF 吗?

<details>
<summary>答案提示</summary>

两种策略:
1. **离散处理**: 直接估计 5 个点的 ATE (类似多臂 Bandit)
2. **连续 DRF + 插值**: 估计连续 DRF,然后在 5 个点上取值

选择取决于:
- 如果离散点足够密集,方法1更稳健
- 如果希望平滑外推,或未来可能增加新值,方法2更灵活
</details>

### 思考题 4: 异质性
不同用户的最优券额可能不同 (高消费用户 vs 低消费用户)。如何扩展到条件 DRF?

<details>
<summary>答案提示</summary>

估计**条件剂量响应函数** (Conditional DRF):
$$
\mu(t, x) = E[Y(t) | X=x]
$$

方法:
1. **分层**: 将用户分组,每组估计独立的 DRF
2. **交互项**: 在模型中加入 $T \times X$ 交互
3. **Causal Forest**: 使用因果森林类方法估计 CATE(t, x)

这是异质性处理效应 (HTE) 的自然扩展。
</details>

---

# 总结

## 核心要点

### 1. 连续处理的特点
- 估计目标从单个数值 (ATE) 变为函数 (DRF)
- 需要广义倾向得分 (GPS) 而非倾向得分
- 复杂度更高,但信息也更丰富

### 2. GPS 方法
- **核心**: 用条件密度 $f(T|X)$ 平衡混淆
- **优势**: 理论清晰,可解释性强
- **局限**: 依赖分布假设 (如正态性)

### 3. DRF 估计方法
| 方法 | 优势 | 劣势 |
|------|------|------|
| GPS 参数 | 简单,可解释 | 可能误设 |
| GPS 非参数 | 灵活 | 数据需求大 |
| DML | 稳健,正交化 | 实现复杂 |

### 4. 业务应用
- **优惠券优化**: 找到最优金额
- **广告预算**: 边际收益递减,ROI 分析
- **定价策略**: 平衡价格与转化率

## 进阶方向

1. **异质性处理效应**: 条件 DRF $\mu(t, x)$
2. **多个处理变量**: 向量处理 $T \in \mathbb{R}^d$
3. **时间动态**: 动态剂量响应
4. **因果强化学习**: 连续动作空间的策略优化

## 推荐阅读

**经典论文**:
- Hirano & Imbens (2004): "The Propensity Score with Continuous Treatments"
- Robinson (1988): "Root-N-Consistent Semiparametric Regression"
- Chernozhukov et al. (2018): "Double/Debiased Machine Learning"

**Python 库**:
- `econml`: 微软的因果推断库,支持连续处理
- `causalml`: Uber 的因果推断工具

---

**恭喜完成本章学习! 🎉**

你已经掌握:
- ✅ 连续处理与二元处理的区别
- ✅ 广义倾向得分的理论与实践
- ✅ 多种 DRF 估计方法
- ✅ 双重机器学习在连续处理中的应用
- ✅ 真实业务场景的建模与优化

下一步: 探索异质性处理效应,学习如何为不同用户定制最优策略!